<a href="https://colab.research.google.com/github/kangning-huang/PopWeightedUHI/blob/main/Population_weighted_SUHI_extremes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap &> /dev/null
!pip install geopandas &> /dev/null

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import geopandas as gdp
import geemap
from geemap import geojson_to_ee, ee_to_geojson

A Google Earth Engine account is required to authenticate [geemap](https://geemap.org/).

In [4]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=r4syl-lUxbaZpFnE8o7hF-i24v7Cy_ra1dIU8YXVv1E&tc=i02wwPAVRdVGUJBvF0lH29ACXb_HscjJJ9OYbaF9UJA&cc=YRHsgUiCRMOaHi3GG7bFwV5T3-Qfm2p_2nOlr9dTYf4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BXbjAJKMYD0B90IXvBbeIxvVjJwTQMwVNzKMtC0JegspsaqG7Eqr50

Successfully saved authorization token.


Download the functional urban areas from [Gobal Human Settlement Layer](https://ghsl.jrc.ec.europa.eu/download.php?ds=FUA).

In [7]:
!curl https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL//GHS_FUA_UCDB2015_GLOBE_R2019A/V1-0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.zip --output GHS_FUA.zip
!unzip GHS_FUA.zip
gdf_FUAs = gdp.read_file('/content/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2987k  100 2987k    0     0  2183k      0  0:00:01  0:00:01 --:--:-- 2183k
